<a href="https://colab.research.google.com/github/love-bees/twitter/blob/master/data_cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

data = pd.read_csv('data.csv', encoding='utf-8')
data.columns = ['id', 'time', 'geo', 'lang', 'hashtags', 'text']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')

x = data['lang'].value_counts(1).head(15)
x.plot(kind='bar',figsize=(10,6),fontsize=13,color='C1')
plt.title('Jezik Tvitera procentualno', fontsize=16)
plt.ylabel('% od svih Tvitova', fontsize=13)
plt.xlabel('Jezik', fontsize=13)

Sređivanje podataka

In [ ]:
#izdvajanje podataka koji su samo na engleskom
text = []
date = []
hashes = []
geo = []
for i, item in enumerate(data['lang']):
    if data['lang'][i] == 'en':
        text.append(data['text'][i])
        date.append(data['time'][i])
        hashes.append(data['hashtags'][i])
        geo.append(data['geo'][i])

In [ ]:
import re

#prečišćavanje podataka
for i in range(len(text)):
    try:
        text[i] = bytes(text[i][2:-2], 'utf-8').decode('unicode_escape')
        hashes[i] = bytes(hashes[i][2:-1], 'utf-8').decode('unicode_escape')
        text[i] = text[i][:text[i].index('https://t.co/')]
        text[i] = text[i].replace("\n", '')
        text[i] = re.sub(r'[^\x00-\x7F]+','', text[i])
    except:
        continue

In [ ]:
#pretvaranje u dataframe
df = pd.DataFrame(columns=['datetime','coordinates','hashes', 'text'])
df['datetime'], df['coordinates'],df['hashes'], df['text'] = [date, geo, hashes, text]
df.head()

Sentiment

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def sentiment_score(text):
    analysis = analyzer.polarity_scores(text)
    return analysis['compound']

analyzer = SentimentIntensityAnalyzer()

df['sentiment_score'] = df['text'].apply(sentiment_score)

df.to_csv('data_cleaned.csv')